In [13]:
from model import atp_graph, losses
from Tutorials.helper import batcher
from data import synthetic_data_gen, feature_extractor
import keras
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import tensorflow as tf
import IPython
import time
import os

from model import atp_pipeline

2023-05-02 16:02:06.089196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
from data import dataset_preparer

#dataset - [weather,traffic,...]

#hyperparams - num_heads,...

In [15]:
weather_processor = dataset_preparer.weather(path_to_weather_data="datasets/weather.csv")

In [16]:
x_train, y_train, x_val, y_val, x_test, y_test = weather_processor()

IndexError: list index out of range

In [15]:
save_dir = "weights/forecasting/weather"

In [ ]:
num_heads = 4

In [16]:
tf.random.set_seed(442)
opt = tf.keras.optimizers.Adam(3e-4)
run=3; heads=2
atp_model = atp_pipeline.atp_pipeline(num_heads=num_heads)
EPOCHS = 2

In [17]:
seq_l = 96+192;
num_batches = 500
batch_s=20
mini = 50000
n_C_te =5

In [18]:
step = 1
tr_step = atp_graph.build_graph()

name_comp = 'run_' + str(run)
logdir = save_dir + '/logs/' + name_comp
folder = save_dir + '/ckpt/check_' + name_comp

if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

In [24]:
for epoch in range(2):
    print("epoch: ",+epoch)
    start = time.time()
    for batch_n in range(10):
         
        x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
        n_C = int(np.random.choice(np.arange(2, 20), 1))
        n_T = 288 - n_C
        y_tr = y
        t_tr = x
        _,_, nll_pp, msex = tr_step(atp_model, opt, t_tr,y_tr,n_C,n_T, training=True)

        if ((batch_n % 2 == 0)):
            
            n_C = 10
            n_T = 200 - n_C
            t_te,y_te = batcher(val_data_scaled[:,:1], val_data_scaled[:,-2:],batch_s = 100)
            μ, log_σ = atp_model([t_te, y_te, 10, 200, False])
            _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
            
        if nll_pp_te < mini:
            mini = nll_pp_te
            IPython.display.clear_output(wait=True)
            print("epoch {} batch {} validation lik pp: {}".format(epoch, batch_n, nll_pp)) 
            manager.save()
            step += 1
            ckpt.step.assign_add(1)
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

epoch:  0


2023-05-02 15:36:47.085272: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [17]:
# nll_list = []; mse_list = []
# num_epochs = 2;num_batches = 10; test_batch_s = 100
# num_runs = 5 ### used to have a std over likelhood results

# for run in range(5):
#     sum_mse_tot = 0; sum_nll_tot = 0
#     tf.random.set_seed(run)
    
#     mini = 50000
    
#     for epoch in range(num_epochs):
#         print("epoch: ",+epoch)
#         start = time.time()
#         for batch_n in range(num_batches):
#             x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
#             n_C = int(np.random.choice(np.arange(2, 20), 1))
#             n_T = 288 - n_C
#             _,_, nll_pp, msex = tr_step(atp_model, opt, x,y,n_C,n_T, training=True)
            
    
    
#     for _ in range(test_data_scaled.shape[0] // test_batch_s):
#         n_C = 10
#         n_T = 200 - n_C
#         t_te,y_te = batcher(test_data_scaled[:,:1], test_data_scaled[:,-2:], batch_s = test_batch_s)
#         μ, log_σ = atp_model([t_te,y_te,10,200, False])
#         _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
#         sum_nll_tot += sum_nll # each sequence is length 190 times 100 seq. per batch 
#         sum_mse_tot += sum_mse
        
#     nllx =  sum_nll_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
#     msex =  sum_mse_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
    
        
#     nll_list.append(nllx.numpy())
#     mse_list.append(msex.numpy())
# np.save(save_dir + '/nll_list.npy', nll_list)    
# np.save(save_dir + '/mse_list.npy', mse_list)    

epoch:  0


2023-04-30 16:59:28.722885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:33.194323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:38.228749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:42.366600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:48.822430: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:53.695908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:04.466689: I tensorflow/core/grappler/optimizers/cust

epoch:  1


2023-04-30 17:00:23.658776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:30.318846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:40.900534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:47.548930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  0


...

2023-04-30 17:06:16.750343: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:06:28.825705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:06:39.023297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:06:45.185655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  0


2023-04-30 17:07:09.548425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:14.081493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:21.297863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:33.094102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:43.536016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:53.878573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:08:05.723638: I tensorflow/core/grappler/optimizers/cust

epoch:  1


2023-04-30 17:08:20.023947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:08:34.902275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:08:46.535644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:09:00.301088: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:09:15.021889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
from model import testing

In [10]:
testing.testing_multiple_times([training_data_scaled, test_data_scaled], save_dir)

Metal device set to: Apple M1 Max
epoch:  0


2023-04-30 20:21:57.231179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-30 20:21:57.231429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-30 20:21:59.990918: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-30 20:21:59.991420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:02.881981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:05.996027: I tensorflow/core/grappler/o

2023-04-30 20:22:15.302552: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2023-04-30 20:22:18.807729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  1


...

2023-04-30 20:26:36.394556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:26:43.921016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:26:49.129320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:26:54.852279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:27:04.047185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  0


2023-04-30 20:27:27.140901: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:27:30.881647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:27:36.311122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:27:43.186670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:27:50.637064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:27:58.104558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:28:03.109396: I tensorflow/core/grappler/optimizers/cust

epoch:  1


2023-04-30 20:28:24.922929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:28:33.284505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:28:39.665855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:28:49.073108: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [41]:
def batcher(t, y, idx_list, batch_s = 32, window = 288):
    '''
    cutting one long array to sequences of length 'window'.
    'batch_s' must be ≤ full array - window length
    '''
    
    
    if len(idx_list) < 1:
        print("warning- you didn't loop over the correct range")
        
    
    batch_s = min(batch_s, y.shape[0]-window)    
    idx = np.random.choice(len(idx_list), batch_s, replace = False)

    y = np.array([np.array(y)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    t = np.array([np.array(t)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    for i in sorted(idx, reverse=True): del idx_list[i]
    return t.squeeze(), y.squeeze(), idx_list

In [34]:
idx_list = list(range(test_data_scaled[:,:1].shape[0] - 288))

In [37]:
a,b,c = batcher(test_data_scaled[:,:1][:, :, np.newaxis], test_data_scaled[:,-2:][:, :, np.newaxis], idx_list, batch_s=100)

In [36]:
test_data_scaled[:,:1].shape

(5270, 1)

In [45]:
len(c)

4882

In [46]:
test_data_scaled[:,:1].shape

(5270, 1)

In [ ]:
for i in range(num_repeats):
    
    # instantiate model
    
    
    
    # train model for num_epochs
        # if loss on val set < mini, save model weights
    
    # reload model from model weights
    
    # evaluate model on test set and append to test set array
    

    


In [ ]:
        
        training_data_scaled, test_data_scaled = inputs
        #instantiation
        
        step = 1
        run=run_num; 
        tf.random.set_seed(run)
        opt = tf.keras.optimizers.Adam(3e-4)
        atp_model = atp_pipeline.atp_pipeline( num_heads=num_heads, projection_shape_for_head=projection_shape_for_head, output_shape=output_shape, rate=rate, permutation_repeats=permutation_repeats,
        bound_std=bound_std, num_layers=num_layers, enc_dim=enc_dim,  xmin=xmin, xmax=xmax)
        tr_step = atp_graph.build_graph()
        name_comp = 'run_' + str(run)
        folder = save_dir + '/ckpt/check_' + name_comp
        if not os.path.exists(folder): os.mkdir(folder)
        ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
        manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
        ckpt.restore(manager.latest_checkpoint) 
        sum_mse_tot = 0; sum_nll_tot = 0
        mini = =50000
        
        #training
        for epoch in range(num_epochs):
            print("epoch: ",+epoch)
            for _ in range(num_batches):
                x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
                n_C = int(np.random.choice(np.arange(2, 20), 1))
                n_T = 288 - n_C
                _,_, _, _ = tr_step(atp_model, opt, x,y,n_C,n_T, training=True)
                
                #saving 
                
                if nll_pp_te < mini:
                    mini = nll_pp_te
                    manager.save()
                    step += 1
                    ckpt.step.assign_add(1)
        
        ckpt.restore(manager.latest_checkpoint) 


        for _ in range(test_data_scaled.shape[0] // test_batch_s):
            n_C = 10
            n_T = 200 - n_C
            t_te,y_te = batcher(test_data_scaled[:,:1], test_data_scaled[:,-2:], batch_s = test_batch_s)
            μ, log_σ = atp_model([t_te,y_te,10,200, False])
            _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
            sum_nll_tot += sum_nll # each sequence is length 190 times 100 seq. per batch 
            sum_mse_tot += sum_mse
            
        nllx =  sum_nll_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
        msex =  sum_mse_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
        
            
        nll_list.append(nllx.numpy())
        mse_list.append(msex.numpy())
    np.save(save_dir + '/nll_list.npy', nll_list)    
    np.save(save_dir + '/mse_list.npy', mse_list)   

In [ ]:
####

#electricity

# make batcher as a function of n_C and n_T - window

## other NP experiments

# test on n_C and n_T which are randomly selected at each point - outside batcher 

# don't repeat testing points  

# for NP experiments, we need to batch differently. for forecasting 100000 x1 ---> 32 x 288 x 1.
# for np, we will have 10000 x 288 x 1 -> 32 x 288 x 1

#also divide mse by n_T each batch for testing

In [ ]:
### set this up for training and saving best models adn then loading it and calculating loglik.

In [50]:
def batcher(t, y, idx_list,batch_s = 32, window = 288,cut_sequence = True):
    '''
    cutting one long array to sequences of length 'window'.
    'batch_s' must be ≤ full array - window length
    '''
    
    
    if len(idx_list) < 1:
        print("warning- you didn't loop over the correct range")
        
    
    batch_s = min(batch_s, y.shape[0]-window)    
    idx = np.random.choice(len(idx_list), batch_s, replace = False)

    y = np.array([np.array(y)[idx_list[i]:idx_list[i]+window] for i in idx])
    t = np.array([np.array(t)[idx_list[i]:idx_list[i]+window] for i in idx])
    for i in sorted(idx, reverse=True): del idx_list[i]
    return t, y,idx_list

In [51]:
y_tr = Y[tr_ix, :, :]
t_tr = t[tr_ix, :]

NameError: name 'Y' is not defined

In [52]:
x_train.shape

(36887, 1)

In [53]:
def batcher(t, y, idx_list, batch_s = 32, window = 288):
    '''
    cutting one long array to sequences of length 'window'.
    'batch_s' must be ≤ full array - window length

    input to forecast: (None, 1, 1) for t,y.
    input to NP tasks: (None, seq_len, 1) for t,y.
    '''
    
    
    if len(idx_list) < 1:
        print("warning- you didn't loop over the correct range")
        
    
    batch_s = min(batch_s, y.shape[0]-window)    
    idx = np.random.choice(len(idx_list), batch_s, replace = False)

    y = np.array([np.array(y)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    t = np.array([np.array(t)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    for i in sorted(idx, reverse=True): del idx_list[i]
        
    t = t.squeeze()
    y = y.squeeze()
    
    if len(t.shape) == 2:
        t = t[:,:,np.newaxis]
        y = y[:,:,np.newaxis]
        
    return t,y, idx_list

In [56]:
idx_list=list(range(y_test.shape[0]-288))

In [58]:
a,b,c = batcher(x_test[:,:,np.newaxis],y_test[:,:,np.newaxis],idx_list=idx_list)

In [60]:
a.shape

(32, 288)

In [61]:
b.shape

(32, 288)

In [64]:
x_dummy = np.random.normal(size=(100,300,1))

In [65]:
y_dummy = np.random.normal(size=(100,300,1))

In [70]:
idx_list=list(range(y_dummy.shape[0]))

In [69]:
y_dummy.shape

(100, 300, 1)

In [71]:
a,b,c = batcher(x_dummy,y_dummy,idx_list=idx_list,window=1)

In [72]:
a.shape

(32, 300)

In [77]:
len(b.shape)

2

In [76]:
len(idx_list)

68